In [220]:
import pandas as pd
import numpy as np
from collections import Counter

In [221]:
data = pd.read_csv('shkib.csv', parse_dates=[0])
data.head()

,_time,src_user,src_ip,src_port,dest_user,dest_ip,dest_port,input_byte,output_byte
0,2018-01-19 16:46:00,c15cf96d9b56740c974661d209ef44f7,9ec3b27794d1d302fa04a94836249f4a,55991,NaN,1f1b55833917801cbd2ed9a313095d9e,443,1019,3562
1,2018-01-19 16:46:00,c15cf96d9b56740c974661d209ef44f7,9ec3b27794d1d302fa04a94836249f4a,55979,NaN,353a9f7687529bc37e3c8bcae6dcfdfa,443,840,357
2,2018-01-19 16:46:00,c15cf96d9b56740c974661d209ef44f7,9ec3b27794d1d302fa04a94836249f4a,34432,NaN,072f83db38ddd013e59a3616bd3b05a6,53,61,135
3,2018-01-19 16:46:00,c15cf96d9b56740c974661d209ef44f7,9ec3b27794d1d302fa04a94836249f4a,49932,NaN,072f83db38ddd013e59a3616bd3b05a6,53,61,77
4,2018-01-19 16:46:00,c15cf96d9b56740c974661d209ef44f7,9ec3b27794d1d302fa04a94836249f4a,55994,NaN,ee76d4dce963e690712339f7bf064705,443,757,3511


In [222]:
# Поиск 5ти пользователей, сгенерировавших наибольшее количество запросов
# идея очень простая: группируем все события по полю src_user,
# затем считаем количество записей в каждой группе
def solve_task1(logs):
    grouped = logs.groupby("src_user").count().sort_values(by=['_time'], ascending=False)
    result = pd.DataFrame(index=grouped.index)
    result['num_of_requests'] = grouped['_time']
    return result[:5]

In [223]:
solve_task1(data)

,num_of_requests
src_user,
a6c732a9cd426b7620cd93e7aa37aef6,54522
7834ea899abe21b5fd047e125cdffc56,50149
64c5eae485561a1d7064477e792a6358,46463
aedf2485753a52ca1149bde2343e8d7a,40371
e948e29d93d507d2bfd6dc2dee45bf2b,37718


In [224]:
# Поиск 5ти пользователей, отправивших наибольшее количество данных
# Данные логи очень похожи на логи прокси сервера. Поэтому отправленные пользователем данные
# это те данные, которые сервер получил, то есть input_bytes
# Здесь идея в следующем: группируем все события по src_user, затем суммируем значения в колонке inpyt_bytes,
# затем сортируем пользователей по количеству отправленных данных (сумме значений в колонке input_bytes в каждой группе)
# в конце возвращаем 5 пользователей с самым большим значением суммы
def solve_task2(logs):
    return logs.groupby("src_user")[['input_byte']].sum().sort_values(by=['input_byte'], ascending=False)[:5]

In [225]:
solve_task2(data)

,input_byte
src_user,
d3263fcb1a81c914d04630c6740cd720,948812255
426cbf7089b22bfc471d3d4446049440,447466106
d7988e912bef788f3ba65a206db8f096,200189029
46a8b3373b4b88c4ab540dcb4cca08a4,161407863
ef96a577df1a391a738b209571551f75,130318688


In [84]:
# Рассматривая события сетевого трафика как символы неизвестного языка,
# найти 5 наиболее устойчивых N-грамм журнала событий (текста на неизвестном языке),
# (https://ru.wikipedia.org/wiki/N-грамм), где N=3-5. 
# Тип символа задается квартетом user+src_port+dest_ip+dest_port.
def solve_task5():
    symbols = get_symbols_from_file("shkib.csv")
    n_3 = get_the_most_popular_n_grams(symbols, 3).most_common(5)
    n_4 = get_the_most_popular_n_grams(symbols, 4).most_common(5)
    n_5 = get_the_most_popular_n_grams(symbols, 5).most_common(5)
    return n_3, n_4, n_5

def event2type(event):
    splited = event.split(",")
    # вставлен символ '_' между каждой из частей чтобы избежать коллизий:
    # например, есть событие с src_user = "fffddd5" и src_port = "555"
    # и событие с src_user = "fffddd" и src_port = "5555"
    # В этом случае как символы они будут эквиваленты. 
    # Чтобы такого не случилось (хотя я не утверждаю что в задании так произойдет)
    # на всякий случай я добавил разделитель, который не используются в полях
    return "{}_{}_{}_{}".format(splited[1], splited[3], splited[5], splited[6])

def get_symbols_from_file(filename="shkib.csv"):
    f = open(filename)
    f.readline() # to get rid of header
    symbols = [event2type(event) for event in f.readlines()]
    f.close()
    return symbols

def get_the_most_popular_n_grams(symbols, n):
    result = [tuple(symbols[i:i+n]) for i in range(0, len(symbols) - n + 1)]
    return Counter(result)

In [232]:
result = solve_task5()

In [258]:
for n, res in zip([3, 4, 5], result):
    print("For n = {}".format(n))
    only_tokens = map(lambda x: x[0], res)
    output = map(lambda x: "\n    ".join(x), only_tokens)
    print("    " + "\n    ------\n    ".join(output))
    print()

For n = 3
    a3e47aed07abf4475dbbc0668abf31c7_38237_072f83db38ddd013e59a3616bd3b05a6_53
    12a561cb1749cf76bf2074d74b05f54f_50606_072f83db38ddd013e59a3616bd3b05a6_53
    12a561cb1749cf76bf2074d74b05f54f_50606_0956783fe5a6d8e0f864b5eaf4084063_53
    ------
    a3e47aed07abf4475dbbc0668abf31c7_38237_072f83db38ddd013e59a3616bd3b05a6_53
    12a561cb1749cf76bf2074d74b05f54f_50606_0956783fe5a6d8e0f864b5eaf4084063_53
    12a561cb1749cf76bf2074d74b05f54f_50606_072f83db38ddd013e59a3616bd3b05a6_53
    ------
    a3e47aed07abf4475dbbc0668abf31c7_38237_0956783fe5a6d8e0f864b5eaf4084063_53
    ec0d21471f0feba0ff59b3e7c2cc548b_43796_072f83db38ddd013e59a3616bd3b05a6_53
    a3e47aed07abf4475dbbc0668abf31c7_38237_072f83db38ddd013e59a3616bd3b05a6_53
    ------
    a3e47aed07abf4475dbbc0668abf31c7_38237_0956783fe5a6d8e0f864b5eaf4084063_53
    a3e47aed07abf4475dbbc0668abf31c7_38237_072f83db38ddd013e59a3616bd3b05a6_53
    12a561cb1749cf76bf2074d74b05f54f_50606_072f83db38ddd013e59a3616bd3b05a6_53
    -----

## Решения для 3-ей и 4-ой задач не рабочие, не стоит их смотреть :)

In [249]:
# Поиск регулярных запросов (запросов выполняющихся периодически) по полю src_user
# под регулярными запросами в этом случае понимаются запросы на один и тот же dest_ip, dest_port и input_byte
def solve_task3(logs, src_user):
    request_uniqueness = ['dest_ip', 'dest_port', 'input_byte']
    requests_per_user = logs[logs['src_user'] == src_user].sort_values(by=['_time'])
    unique_requests_indices = get_unique_requests_indices(requests_per_user, request_uniqueness)
    req = get_unique_request_by_index(requests_per_user, unique_requests_indices[4])
    return req
    return unique_requests_indices
    #return requests_per_user.sort_values(by=['_time'])

class ReqGroup:
    def __init__():
        pass

def get_unique_requests_indices(requests_per_user, request_uniqueness):
    index = requests_per_user.groupby(request_uniqueness).count().index
    result = []
    for item in zip(*index.labels):
        r = dict()
        for i, name in enumerate(index.names):
            r[name] = index.levels[i][item[i]]
        result.append(r)
    return result
        
def get_unique_request_by_index(requests, index_item):
    result = requests
    for param, value in index_item.items():
        result = result[result[param] == value]
    return result

def extract_period_for_unique_request(requests):
    pass

In [255]:
t = solve_task3(data, "a78020d74cd36c56bd0f4597ee081957")

In [253]:
# Поиск регулярных запросов (запросов выполняющихся периодически) по полю src_ip
# Здесь логика абсолютно такая-же, которая была и в задании три, только вместо src_user используется src_ip
def solve_task4(logs):
    grouped = logs.groupby(["src_user", "input_byte", "dest_port", "dest_user", "dest_ip"]).count().sort_values(by=['_time'], ascending=False)
    result = pd.DataFrame(index=grouped.index)
    result['count'] = grouped['_time']
    return result[result['count'] > 10]

In [254]:
solve_task4(data)

,,,,,count
src_user,input_byte,dest_port,dest_user,dest_ip,
a78020d74cd36c56bd0f4597ee081957,300,631,e948e29d93d507d2bfd6dc2dee45bf2b,3f640ac7c2b8014b9601a80c5ccd1df2,80
f2e55a84ab0a0605c25640a225bb050c,120,631,e948e29d93d507d2bfd6dc2dee45bf2b,3f640ac7c2b8014b9601a80c5ccd1df2,67
acacb13ab8a3ed3841d15ed3dc1d24d9,300,631,e948e29d93d507d2bfd6dc2dee45bf2b,3f640ac7c2b8014b9601a80c5ccd1df2,63
6ebbd39864f9f660d691392b16f09e3f,594,631,0f0c129694085ab9c5ad2e7a9c28378c,ddc69fecac929dce4a20d790a587fc7b,56
acacb13ab8a3ed3841d15ed3dc1d24d9,300,631,0f0c129694085ab9c5ad2e7a9c28378c,ddc69fecac929dce4a20d790a587fc7b,55
a86356248128c94c718633744eb51988,300,631,0f0c129694085ab9c5ad2e7a9c28378c,ddc69fecac929dce4a20d790a587fc7b,45
eb77f08a94216f89f5b644f31dda051b,120,631,0f0c129694085ab9c5ad2e7a9c28378c,ddc69fecac929dce4a20d790a587fc7b,39
47c05a1b981e422f7ac2501d1b8d5424,300,631,e948e29d93d507d2bfd6dc2dee45bf2b,3f640ac7c2b8014b9601a80c5ccd1df2,36
37528b947bd184c90383768abfacd9a4,300,631,0f0c129694085ab9c5ad2e7a9c28378c,ddc69fecac929dce4a20d790a587fc7b,35
